#Input Guardrails


In [1]:
!pip install -Uq openai-agents pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.6/734.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00


In [2]:

import nest_asyncio
nest_asyncio.apply()

In [6]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
    output_guardrail,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)
from google.colab import userdata


gemini_api_key = userdata.get("GOOGLE_API_KEY")

if not gemini_api_key:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable")

In [7]:
class MathHomeworkOutput(BaseModel):
  is_math_homework : bool # Boolean Value represents decision
  reasoning : str # Showe the reasoning for the decision
  answer: str # This is the answer to the user question

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)


guardrail_agent = Agent(
    name="Guardrail Check",
    instructions="Check if the user is asking you to do their math homeowrk",
    output_type=MathHomeworkOutput,
    model=model
)

result = Runner.run_sync(guardrail_agent, "What is the capital of pakistan")

print(result.final_output)

is_math_homework=False reasoning='This is a question about geography, not mathematics.' answer='Islamabad'


In [14]:
result = Runner.run_sync(guardrail_agent, "What is 2 + 2")

print(result.final_output.answer)
print(result.final_output.is_math_homework)
print(result.final_output.reasoning)

4
True
This is a basic arithmetic question that is commonly found in math homework.


In [12]:
@input_guardrail
async def math_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)
    print(f"""[Guardrail Debugging] , The response is
    {result.final_output} """)
    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math_homework,
    )
agent = Agent(
    name="Math HOmeowrk Assistant",
    instructions="You are a customer support agent. You help customers with their questions.",
    input_guardrails=[math_guardrail],
    model=model
)


try:
    await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
    print("Guardrail didn't trip - this is unexpected")

except InputGuardrailTripwireTriggered:
    print("Math homework guardrail tripped")






[Guardrail Debugging] , The response is 
    is_math_homework=True reasoning='The user is asking for help with a simple algebraic equation, which is a common type of math homework problem. The equation 2x + 3 = 11 can be solved by subtracting 3 from both sides to get 2x = 8, then dividing both sides by 2 to get x = 4.' answer='x = 4' 
Math homework guardrail tripped


# Output Guardrails

In [20]:
class MessageOutput(BaseModel):
    response: str

class MathOutput(BaseModel):
    is_math: bool
    reasoning: str

guardrail_agent2 = Agent(
    name="Guardrail check",
    instructions="Check if the output includes any math.",
    output_type=MathOutput,
)

@output_guardrail
async def math_guardrail2(
    ctx: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent2, output.response, context=ctx.context, run_config = config)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )

agent2 = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[math_guardrail2],
    output_type=MessageOutput,
)


# This should trip the guardrail
try:
    await Runner.run(agent2, "Hello, can you help me solve for x: 2x + 3 = 11?", run_config = config)
    print("Guardrail didn't trip - this is unexpected")

except OutputGuardrailTripwireTriggered:
    print("Math output guardrail tripped")

Math output guardrail tripped
